# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [1]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图片展示了一片美丽的自然景观。图中的主体是一条沿着高草覆盖的地面铺设的木板路，引导观者的视线深入画面。这条小径两侧是茂盛的绿色草地，显得生机勃勃。背景中可以看到一些灌木丛和树木，天空是广阔的，有几朵淡淡的云彩，呈现出宁静美好的夏日气息。\n\n整体上，这幅图传递了一种平和与自然未受破坏的感觉，是很多人在日常忙碌生活后寻求放松和平静的理想场所。这样的自然景观可以激发人们对户外活动的兴趣，并提醒我们保护和珍惜自然环境的重要性。', role='assistant', function_call=None, tool_calls=None))


In [2]:
response.choices[0].message.content

'这幅图片展示了一片美丽的自然景观。图中的主体是一条沿着高草覆盖的地面铺设的木板路，引导观者的视线深入画面。这条小径两侧是茂盛的绿色草地，显得生机勃勃。背景中可以看到一些灌木丛和树木，天空是广阔的，有几朵淡淡的云彩，呈现出宁静美好的夏日气息。\n\n整体上，这幅图传递了一种平和与自然未受破坏的感觉，是很多人在日常忙碌生活后寻求放松和平静的理想场所。这样的自然景观可以激发人们对户外活动的兴趣，并提醒我们保护和珍惜自然环境的重要性。'

### 封装成一个函数 query_image_description

In [3]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI()  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [4]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图是一种幽默风格的搞笑比较。图中展示了两种不同状态的狗。左边是一只被幻想成拥有非常发达肌肉、类似人类健美运动员身体的柴犬，配文“16岁的我，工作后的我”，表达了年轻时充满活力和梦想的状态。右边的柴犬则显得普通、略显憔悴，配有文字“好累好困、好想摸鱼、看到鸡腿就开心，不属于我、我也有小肚腩、肉肉的很可爱”，描绘了工作后可能感到疲惫、减少锻炼而略显发福的现实状态。

这种对比通常用来幽默地表达人们对于理想与现实之间差距的感慨，展示了许多人对于年轻时的憧


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [6]:
from openai import OpenAI
import base64
import requests
import json
import os

client = OpenAI()  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    openai_base_url = os.getenv("OPENAI_BASE_URL")
    response = requests.post(f"{openai_base_url}/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [6]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

这幅图展示了1980年到2020年间，美国、中国、日本和德国的国内生产总值（GDP）比较。每个国家的GDP以万亿美元为单位标出。从图中可以看出：

- **美国（蓝线）**：GDP持续增长，从1980年的约3万亿美金增长到2020年的超过20万亿美金。
- **中国（红线）**：自1980年起，GDP增长非常显著，从不到1万亿美金增长至接近15万亿美金。
- **日本（紫线）**：1980年至1995年GDP增长明显，之后增长放缓，并在2005年至2015年间保持相对平稳。
- **德国（绿线）**：GDP增长较为平稳，从1980年的不到1万亿美金增长至2020年的约4万亿美金。

总体而言，图表清晰地表示了这四个经济体在过去四十年间的经济增长情况，其中中国的增长尤其引人注目，显示了其经济的迅速崛起。美国则保持了其全球经济领导地位的增长趋势。日本在90年代后增长放缓，而德国则显示出稳定的增长模式。


#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [10]:
content = query_base64_image_description("./images/handwriting_0.jpg")
print(content)

这张图片显示的是一本笔记本上手写的文字，内容主要涉及到自然语言处理（NLP）中的训练技术，如Prompt Tuning和LoRA技术。

1. **Prompt Tuning（简要模型调整）**： 提到了使用Prompt Tuning来调整一个小型的Transformer模型。此处解释了输入X（由个别输入X1, X2, ..., Xn组成）。每个输入首先通过一个Embedding过程转换，然后通过Token变换。输出Y是通过矩阵W与转换后的输入X'之间的乘法得出。

2. **Prefix Tuning：** 这部分说明了Prefix Tuning的过程，其中添加了前缀权重W_p到原始权重W_j中，得到新的权重W'用于生成输出Y。

3. **LoRA调整技术**： 这部分涉及Linear Re-parameterization（线性重新参数化），通过调整矩阵ΔW（通过两个矩阵A和B的乘积表示）来修改权重W。这是一种节省参数调整的方法，使原有模型的W变为W+ΔW，这里也涉及到了一些矩阵运算和优化策略。

其中还提到了两个案例分析的存储需求：“LLAMA”需要65GB，而经过LoRA调整的“QLLoRA”仅需要48GB。

这些笔记对于理解NLP中一些先进的模型调整技术十分有用，尤其对于需要在资源受限的环境下部署NLP模型的研究人员或实践者。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [9]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

这张图片展示了一张写有每天日程的纸。上面写着“活动事务安排”，列出了一周的部分活动安排，分别是星期一到星期五的计划：

- 星期一：
  - 6:30 起床
  - 8:00 早餐
  - 10:00 听口语课

- 星期二：
  - 2:00 领取成绩单和毕业证
  - 5:00 拿钥匙之前走
  - 9:00 校车

这张纸条可能是为了帮助某人规划和记忆一周中特定的活动和任务。

![](./images/handwriting_1.jpg)

In [12]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

这张图片展示的是一本笔记本的两页，内容涉及深度学习、特别是关于自然语言处理（NLP）的各种技术和方法。主要讨论了Transformer模型及其改进方法和训练技术。

左侧页面的上部标注有“自然语言处理”、“基础”和“评价”，可能是对内容的分类。提到了Transformer模型，并列举了不同的测试标准和指标，如PeFT (“Prompt-based Fine-Tuning”) 和模型性能对比（“Benchmark”）。此外，还提到了不同的方法，如Prompt Tuning和Adapter。具体包括：  
- Adapter: 一个2019年Google的研究
- Prefix: 代表2021年Stanford的工作
- Prompt: 同样是2021年Google的研究
- P-Tuning V1和V2：2021年的两种方法
- Soft prompts：2021年的研究，提示模板基于模板

右侧页面讨论了多模态指令式微调（multi-modality instruction FT）、Llama (3B)、LoRA、PETC（2022年的新技术）等。还有部分文字描述了如何使用prefix-tuning和Adapter方法来细化在大型语言模型（LLMs）中的处理。

页面提到了几种语言模型，如：
- Llama 
- BLOOM
- ChatGLM 
- Alpaca

这些内容表明这本笔记本的主人正在研究或学习NLP领域的最新技术和方法，特别是如何通过各种微调技术提升已有的大型语言模型的性能。

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。

# #1 使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果

![](./images/edward's-todo-list.jpg)

In [7]:
content = query_base64_image_description("./images/edward's-todo-list.jpg")
print(content)

这张图片展示了一张写有每天日程的纸。上面写着“活动事务安排”，列出了一周的部分活动安排，分别是星期一到星期五的计划：

- 星期一：
  - 6:30 起床
  - 8:00 早餐
  - 10:00 听口语课

- 星期二：
  - 2:00 领取成绩单和毕业证
  - 5:00 拿钥匙之前走
  - 9:00 校车

这张纸条可能是为了帮助某人规划和记忆一周中特定的活动和任务。


In [10]:
display(Markdown(content))

这张图片展示了一张写有每天日程的纸。上面写着“活动事务安排”，列出了一周的部分活动安排，分别是星期一到星期五的计划：

- 星期一：
  - 6:30 起床
  - 8:00 早餐
  - 10:00 听口语课

- 星期二：
  - 2:00 领取成绩单和毕业证
  - 5:00 拿钥匙之前走
  - 9:00 校车

这张纸条可能是为了帮助某人规划和记忆一周中特定的活动和任务。

# 个人总结

- 可以识别出内容主题是 todo list
- 精确识别出各项任务的时间节点
- 中文识别准确率非常低

# #2 整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。

In [16]:
def query_base64_image_description_v2(image_path, max_tokens=1000):
    content = query_base64_image_description(image_path, prompt="请使用 Markdown 格式解释下图里的内容？", max_tokens=max_tokens)
    display(Markdown(content))
    
query_base64_image_description_v2("./images/edward's-todo-list.jpg")

这张图片上是一张写有每周日程的纸片。纸片分为“上午”和“下午”两部分，列出了时间及对应的活动。

### 上午
- **6:30** - 起床
- **8:00** - 跑步
- **10:00** - 听力练习

### 下午
- **2:00** - 讨论比赛用品
- **5:00** - 锻炼背部肌肉
- **9:00** - 睡觉

上方标题：“活力充沛课程”，可能意味着这是一种健康或训练的日程安排。